In [4]:
#from netCDF4 import Dataset
import cartopy.crs as ccrs

import matplotlib.pyplot as plt
import numpy as np
import scipy.io


## The following cell Loads all the data necessary for the question

### All data file description
<strong>'treecovdata.nc'</strong> contains the datastructure that can plot the world map

<strong>'2023worldforestdata.mat'</strong>  contains all the past and current rainfall data (1999, 2014, 2100), as well as the past forest cover (1999). Moreover, this file also contains the latitudes and longitudes of the world map for plotting. Note: latitude and longitudes are described as (1 X N) array but in the cell below we convert it into a squeezed array

<strong> worldfixedpointdata.mat </strong> file contains the hysterisis/fixedpoint data of 3 continents.

The variable named <strong> fixedpointdata_xyz </strong>as a (K X 4) numpy array. 'K' represents the total number of fixed points. Each row contains the following 4 columns:
- column 1: the amount of rainfall
- column 2: minimum stable fixed point
- column 3: maximum stable fixed point (if none then it contains 'Nan')
- column 4: the unstable fixed point (if none then it contains 'Nan')

In [6]:
fname = './treecovdata.nc'
fname_mat = './2023worldforestrainfalldata.mat'
mat_file = scipy.io.loadmat(fname_mat)

rainfall1999 = mat_file['rainfall1999']
rainfall2014 = mat_file['rainfall2014']
rainfall2100 = mat_file['rainfall2100']

forest1999 = mat_file['forest1999']
lats = np.squeeze(mat_file['lats'], axis=0)
lons = np.squeeze(mat_file['lons'],axis=0)

In [ ]:
fname_mat = './worldfixedpointdata.mat'
mat_file = scipy.io.loadmat(fname_mat)
fixedpointdata_southamerica = mat_file['fixedpointdata_southamerica']
fixedpointdata_africa = mat_file['fixedpointdata_africa']
fixedpointdata_aus = mat_file['fixedpointdata_aus']

## Above code has loaded all variables of interest into the given variables:

- rainfall1999
- rainfall2014
- rainfall2100

- forest1999
- lats
- lons
- fixedpointdata_southamerica
- fixedpointdata_africa
- fixedpointdata_aus

## Plotting the equilibrium/hysterisis curve of the three continents (from fixedpointdata_XYZ variables)

 - stable points are denoted by circle marker
 - unstable points are denoted by (+) markers

In [ ]:
plt.figure(figsize=(10, 3))
for i in range(fixedpointdata_aus.shape[0]):
    plt.subplot(1,3,1) # South america
    plt.plot(fixedpointdata_southamerica[i,0], fixedpointdata_southamerica[i,1], 'k.')
    plt.plot(fixedpointdata_southamerica[i,0], fixedpointdata_southamerica[i,2], 'k.')
    plt.plot(fixedpointdata_southamerica[i,0], fixedpointdata_southamerica[i,3], 'k+')
    plt.ylabel('tree cover (%)')
    plt.xlabel('rainfall (mm/yr)')
    
    plt.subplot(1,3,2) # Africa 
    plt.plot(fixedpointdata_africa[i,0], fixedpointdata_africa[i,1], 'k.')
    plt.plot(fixedpointdata_africa[i,0], fixedpointdata_africa[i,2], 'k.')
    plt.plot(fixedpointdata_africa[i,0], fixedpointdata_africa[i,3], 'k+')
    plt.xlabel('rainfall (mm/yr)')
    
    plt.subplot(1,3,3) # Australasia
    plt.plot(fixedpointdata_aus[i,0], fixedpointdata_aus[i,1], 'k.')
    plt.plot(fixedpointdata_aus[i,0], fixedpointdata_aus[i,2], 'k.')
    plt.plot(fixedpointdata_aus[i,0], fixedpointdata_aus[i,3], 'k+')
    plt.xlabel('rainfall (mm/yr)')

## To separately compute forest cover on each continent, we are providing the coordiantes of the 3 continents. These three continents vary only in their range of longitudes while the latitudes are constant across the data.

In [ ]:
# longitudes of southamerica, africa and australia [lower boundary value, higher boundary value]
lons_sa = [-90, -30] # South america coordinates (longitude range)
lons_af = [-30, 60] # African coordinates (longitude range)
lons_au = [60, 160] # Australia coordinates (longitude range)

            

In [ ]:
# Here compute the forest cover in south america, given the rainfall amount at 2100 and the hysterisis/equilibrium curve fixedpointdata_southamerica
# Place the computed forest cover in the variable named 'forest2100'








# NOTE: for computing the forest cover of south america, ignore what happens to the rest of the world. i.e., the rest-of-the-world map forest cover can be zeros or nan values

## The cell below plots the forest cover in south america from the value provided in forest2100

In [ ]:
# forest data 2100
fig = plt.figure(figsize=(20, 20))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([min(lons), max(lons), min(lats) , max(lats)], crs=ccrs.PlateCarree())
cs=plt.contourf(lons, lats, forest1999, cmap='Greens')  
#fig.colorbar(cs)
plt.title('forest cover in America in the year 1999 is:', fontname="Times New Roman", fontweight="bold")
ax.coastlines()   
    

fig = plt.figure(figsize=(20, 20))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([min(lons), max(lons), min(lats) , max(lats)], crs=ccrs.PlateCarree())
cs=plt.contourf(lons, lats, forest2100, cmap='Reds', vmin=0, vmax=80)  
#fig.colorbar(cs)
plt.title('forest cover in America in the year 1999 is:', fontname="Times New Roman", fontweight="bold")
ax.coastlines()   

In [ ]:
# Here compute the forest cover in africa, given the rainfall amount at 2100 and the hysterisis/equilibrium curve fixedpointdata_africa
# Place the computed forest cover in the variable named 'forest2100'








# NOTE: for computing the forest cover of africa, ignore what happens to the rest of the world. . i.e., the rest-of-the-world map forest cover can be zeros or nan values

## The cell below plots the forest cover in africa from the value provided in forest2100

In [ ]:
# forest data 2100
fig = plt.figure(figsize=(20, 20))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([min(lons), max(lons), min(lats) , max(lats)], crs=ccrs.PlateCarree())
cs=plt.contourf(lons, lats, forest1999, cmap='Greens')  
#fig.colorbar(cs)
plt.title('forest cover in Africa in the year 1999 is:', fontname="Times New Roman", fontweight="bold")
ax.coastlines()   
    

fig = plt.figure(figsize=(20, 20))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([min(lons), max(lons), min(lats) , max(lats)], crs=ccrs.PlateCarree())
cs=plt.contourf(lons, lats, forest2100, cmap='Reds', vmin=0, vmax=80)  
#fig.colorbar(cs)
plt.title('forest cover in Africa in the year 1999 is:', fontname="Times New Roman", fontweight="bold")
ax.coastlines()   

In [ ]:
# Here compute the forest cover in australia+asia (australasia), given the rainfall amount at 2100 and the hysterisis/equilibrium curve fixedpointdata_aus
# Place the computed forest cover in the variable named 'forest2100'








# NOTE: for computing the forest cover of australasia, ignore what happens to the rest of the world. i.e., the rest-of-the-world map forest cover can be zeros or nan values

In [ ]:
# forest data 2100
fig = plt.figure(figsize=(20, 20))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([min(lons), max(lons), min(lats) , max(lats)], crs=ccrs.PlateCarree())
cs=plt.contourf(lons, lats, forest1999, cmap='Greens')  
#fig.colorbar(cs)
plt.title('forest cover in Australasia in the year 1999 is:', fontname="Times New Roman", fontweight="bold")
ax.coastlines()   
    

fig = plt.figure(figsize=(20, 20))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([min(lons), max(lons), min(lats) , max(lats)], crs=ccrs.PlateCarree())
cs=plt.contourf(lons, lats, forest2100, cmap='Reds', vmin=0, vmax=80)  
#fig.colorbar(cs)
plt.title('forest cover in Australasia in the year 1999 is:', fontname="Times New Roman", fontweight="bold")
ax.coastlines()   